In [1]:
import time
import datetime
import re
import csv

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import pandas as pd
from openpyxl import load_workbook

import configparser 
config = configparser.ConfigParser()
config.read('config.ini')
ip = config['DEFAULT']['IP']
port = config['DEFAULT']['MongoDB-Port']

from pymongo import MongoClient
client = MongoClient(ip, int(port))

import twitter

In [2]:
#get value from config file
CK = config['TWITTER']['CK']
CS = config['TWITTER']['CS']
ATK = config['TWITTER']['ATK']
ATS = config['TWITTER']['ATS']
rank = int(config['TWITTER']['Rank'])

# set to extended mode and sleep if exceed rate limit
api = twitter.Api(consumer_key=CK,
                  consumer_secret=CS,
                  access_token_key=ATK,
                  access_token_secret=ATS,
                  tweet_mode= 'extended',
                  sleep_on_rate_limit=True)

In [3]:
db_twitter = client["Twitter"]
collections_twitter = db_twitter.collection_names()

In [4]:
# get current year and week
current_timestamp = int(time.time() * 1000)
current_year = int(datetime.datetime.now().year)
print("current year : " + str(current_year))

current_week = int((current_timestamp - 1546214400000)/1000/604800)+1
print("current week : " + str(current_week))

current year : 2019
current week : 11


In [5]:
dic_collection = {}
for i in collections_twitter:
    if i.startswith("20") and "Australia" in i:
        year = i[0:4]
        week = re.search('_(.+?)_', i).group(1)[1:]
        if int(year) < current_year:
            pass
            #dic_collection[i] = "{:,}".format(db_twitter[i].find({}).count())
        else:
            try:
                if int(week) < current_week:
                    dic_collection[i] = "{:,}".format(db_twitter[i].find({}).count())
            except: pass

for key in sorted(dic_collection):
    print("%s: %s" % (key, dic_collection[key]))

2019_W10_Twitter_Australia: 40,767
2019_W1_Twitter_Australia: 40,880
2019_W2_Twitter_Australia: 37,645
2019_W3_Twitter_Australia: 52,348
2019_W4_Twitter_Australia: 59,625
2019_W5_Twitter_Australia: 61,617
2019_W6_Twitter_Australia: 61,435
2019_W7_Twitter_Australia: 28,017
2019_W8_Twitter_Australia: 60,035
2019_W9_Twitter_Australia: 58,316


In [6]:
# read keywords from csv file
with open('supporting-files/keywords.csv', 'r') as f:
    search_terms = [row[0] for row in csv.reader(f)]
print("keywords: " + str(search_terms))

keywords: ['#metoo', '#timesup', '#violenceagainstwomen', '#genderequality', '#domesticviolence']


In [ ]:
for e in sorted(dic_collection)[rank::4]:
    if "Australia" in e:
        print("--------------\n" + e)
        c = 0
        # get the all records from "twitter-australia" collection
        for document in db_twitter[e].find({"truncated":True,"soda_checked_truncated_utc":{"$exists":False}},no_cursor_timeout = True):
            c += 1
            try:
                # get full_text using twitter api
                full_text = api.GetStatus(document["id_str"]).full_text
                print(str(c) + "--------------\n" + "got full_text")
                
                t = int(time.time())
                # update fields(add full_text if exist and change truncated value to false)
                db_twitter[e].update({"_id": document["_id"]}, {"$set": {"text": full_text, "truncated":False , "soda_checked_truncated_utc": t }})
            except:
                
                t = int(time.time())
                db_twitter[e].update({"_id": document["_id"]}, {"$set": {"soda_checked_truncated_utc": t}})
                print(str(c) + "--------------\n" + "pass except")
                    
        print("--------------\n" + "Done: get full_text of " + str(c) + " truncated tweets in collection " + e)
    elif "Other" in e:
        print("--------------\n" + e)
        c = 0
        for term in search_terms:
            for document in db_twitter[e].find({"$text": {"$search": "\"" + term + "\"", "$caseSensitive": False},"truncated":True,"soda_checked_truncated_utc":{"$exists":False}},no_cursor_timeout = True):
                c+=1
                try:
                    # get full_text using twitter api
                    full_text = api.GetStatus(document["id_str"]).full_text
                    print(str(c) + "--------------\n" + "got full_text")

                    t = int(time.time())
                    # update fields(add full_text if exist and change truncated value to false)
                    db_twitter[e].update({"_id": document["_id"]}, {"$set": {"text": full_text, "truncated":False , "soda_checked_truncated_utc": t }})
                except:
                    t = int(time.time())
                    db_twitter[e].update({"_id": document["_id"]}, {"$set": {"soda_checked_truncated_utc": t}})
                    print(str(c) + "--------------\n" + "pass except")

        print("--------------\n" + "Done: get full_text of " + str(c) + " truncated keyword tweets in collection " + e)        